# Python Data Science

> Project: Taiwan Election 2020

Kuo, Yao-Jen <yaojenkuo@datainpoint.com> from [DATAINPOINT](https://www.datainpoint.com/)

## TL; DR

> In this lecture, we will talk about how to wrangle the election data of Taiwan in 2020 with `pandas`.

In [1]:
# Importing libraries/functions the project needs
from urllib.parse import quote_plus
from string import ascii_uppercase
import os
import re
import pandas as pd
import sqlite3

## Importing Data

## Data comes from www.cec.gov.tw

- [第15任總統(副總統)選舉](https://db.cec.gov.tw/histMain.jsp?voteSel=20200101A1)
- [第10屆立法委員選舉](https://db.cec.gov.tw/histMain.jsp?voteSel=20200101A2)

## The original data format is spreadsheet

We can use `pd.read_excel` for importing.

## Importing the presidential data of 臺北市

- We use `quote_plus` function from `urllib.parse` to transform file name into a URL
- Check the list of spreadsheets before importing

In [2]:
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


In [3]:
df = pd.read_excel(spreadsheet_url)
df.head()

,第15任總統副總統選舉候選人在臺北市各投開票所得票數一覽表,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,鄉(鎮、市、區)別,村里別,投票所別,各組候選人得票情形,NaN,NaN,有效票數A\nA=1+2+...+N,無效票數B,投票數C\nC=A+B,已領未投票數\nD\nD=E-C,發出票數E\nE=C+D,用餘票數F,選舉人數G\nG=E+F,投票率H\nH=C÷G
1,NaN,NaN,NaN,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,總 計,NaN,NaN,"70,769","685,830","875,854","1,632,453","21,381","1,653,834",143,"1,653,977","513,287","2,167,264",76.3098


## Importing the legislative data of 臺北市

- The regional legislative spreadsheets are a bit more complicated than other spreadsheets
- Check the list of spreadsheets before importing

In [4]:
file_name = "區域立委-A05-2-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市第1選舉區', '臺北市第2選舉區', '臺北市第3選舉區', '臺北市第4選舉區', '臺北市第5選舉區', '臺北市第6選舉區', '臺北市第7選舉區', '臺北市第8選舉區']


## We can specify which sheet to import via `sheet_name`

In [5]:
regional_legislator_taipei_city = {}
for sheet_name in xl.sheet_names:
    regional_legislator_taipei_city[sheet_name] = pd.read_excel(spreadsheet_url, sheet_name=sheet_name)
for k, v in regional_legislator_taipei_city.items():
    print("Shape of {} : {}".format(k, v.shape))

Shape of 臺北市第1選舉區 : (219, 18)
Shape of 臺北市第2選舉區 : (224, 17)
Shape of 臺北市第3選舉區 : (233, 15)
Shape of 臺北市第4選舉區 : (248, 18)
Shape of 臺北市第5選舉區 : (225, 19)
Shape of 臺北市第6選舉區 : (200, 21)
Shape of 臺北市第7選舉區 : (222, 16)
Shape of 臺北市第8選舉區 : (212, 20)


In [6]:
file_name = "不分區立委-A05-6-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


In [7]:
file_name = "山地立委-A05-4-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


In [8]:
file_name = "平地立委-A05-4-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


## Wrangling Data

## Once the import part is done, it is time to wrangle the messy spreashets into a tidy data format

> Tidy datasets are all alike, but every messy dataset is messy in its own way
>
> [Hadley Wickham](http://hadley.nz/)

## What is tidy data?

> There are three interrelated rules which make a dataset tidy:
> 
> 1. Each variable must have its own column.
> 2. Each observation must have its own row.
> 3. Each value must have its own cell.

Source: <https://vita.had.co.nz/papers/tidy-data.pdf>

## Why tidy data?

Simply put, once our data is in tidy format, it is easy to use and quite convenient to tranform into the designated format of visualization or modeling functions for the next stage of data science project.

## What makes our original spreadsheets messy?

- Combined cells
- Missing values
- Summations within observations
- Value in variable names

## Using `skiprows` to skip those combined cells while importing spreadsheets

In [9]:
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')

In [10]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,總 計,NaN,NaN,70769,685830,875854,1632453,21381,1653834,143,1653977,513287,2167264,76.3098
1,松山區,NaN,NaN,5436,55918,64207,125561,1762,127323,2,127325,37329,164654,77.3276
2,NaN,莊敬里,573.0,36,391,429,856,14,870,0,870,230,1100,79.0909
3,NaN,莊敬里,574.0,46,382,438,866,12,878,0,878,259,1137,77.2208
4,NaN,莊敬里,575.0,48,393,389,830,22,852,0,852,262,1114,76.4811


## Updating `columns` attributes with new columns

In [11]:
n_cols = df.columns.size
n_candidates = n_cols - 11
id_vars = ['town', 'village', 'office']
candidates = list(df.columns[3:(3 + n_candidates)])
office_cols = list(ascii_uppercase[:8])
col_names = id_vars + candidates + office_cols
df.columns = col_names

In [12]:
print(n_candidates)
print(candidates)
print(office_cols)
print(col_names)

3
['(1)\n宋楚瑜\n余湘', '(2)\n韓國瑜\n張善政', '(3)\n蔡英文\n賴清德']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
['town', 'village', 'office', '(1)\n宋楚瑜\n余湘', '(2)\n韓國瑜\n張善政', '(3)\n蔡英文\n賴清德', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']


## Using `fillna` to forward-fill `district` values

`ffill`: propagate last valid observation forward to next valid.

In [13]:
filled_towns = df['town'].fillna(method='ffill')
df = df.assign(town=filled_towns)
df.head()

,town,village,office,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,A,B,C,D,E,F,G,H
0,總 計,NaN,NaN,70769,685830,875854,1632453,21381,1653834,143,1653977,513287,2167264,76.3098
1,松山區,NaN,NaN,5436,55918,64207,125561,1762,127323,2,127325,37329,164654,77.3276
2,松山區,莊敬里,573.0,36,391,429,856,14,870,0,870,230,1100,79.0909
3,松山區,莊敬里,574.0,46,382,438,866,12,878,0,878,259,1137,77.2208
4,松山區,莊敬里,575.0,48,393,389,830,22,852,0,852,262,1114,76.4811


## Removing summations via `dropna`

- To comply with the tidy data rule
- To avoid wrong summations

In [14]:
df = df.dropna()
df.head()

,town,village,office,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,A,B,C,D,E,F,G,H
2,松山區,莊敬里,573.0,36,391,429,856,14,870,0,870,230,1100,79.0909
3,松山區,莊敬里,574.0,46,382,438,866,12,878,0,878,259,1137,77.2208
4,松山區,莊敬里,575.0,48,393,389,830,22,852,0,852,262,1114,76.4811
5,松山區,莊敬里,576.0,43,389,462,894,14,908,0,908,271,1179,77.0144
6,松山區,東榮里,577.0,38,431,545,1014,18,1032,0,1032,272,1304,79.1411


## Removing extra spaces in `town` via `replace`

In [15]:
print(df['town'].unique())
stripped_town = df['town'].str.replace("\u3000", "")
df = df.assign(district=stripped_town)
print(df['town'].unique())

['\u3000松山區' '\u3000信義區' '\u3000大安區' '\u3000中山區' '\u3000中正區' '\u3000大同區'
 '\u3000萬華區' '\u3000文山區' '\u3000南港區' '\u3000內湖區' '\u3000士林區' '\u3000北投區']
['\u3000松山區' '\u3000信義區' '\u3000大安區' '\u3000中山區' '\u3000中正區' '\u3000大同區'
 '\u3000萬華區' '\u3000文山區' '\u3000南港區' '\u3000內湖區' '\u3000士林區' '\u3000北投區']


## Pivoting candidate columns into a new pair of variables via `melt`

In [16]:
df = df.drop(labels=office_cols, axis=1)
df_long = pd.melt(df,
                  id_vars=id_vars,
                  var_name='candidate_info',
                  value_name='votes'
                 )
df_long.head()

,town,village,office,candidate_info,votes
0,松山區,莊敬里,573.0,(1)\n宋楚瑜\n余湘,36
1,松山區,莊敬里,574.0,(1)\n宋楚瑜\n余湘,46
2,松山區,莊敬里,575.0,(1)\n宋楚瑜\n余湘,48
3,松山區,莊敬里,576.0,(1)\n宋楚瑜\n余湘,43
4,松山區,東榮里,577.0,(1)\n宋楚瑜\n余湘,38


## Define a function called `tidy_dataframe` which assembles previous wrangling codes

In [17]:
def tidy_dataframe(df):
    # updating columns attributes 
    n_cols = df.columns.size
    n_candidates = n_cols - 11
    id_vars = ['town', 'village', 'office']
    candidates = list(df.columns[3:(3 + n_candidates)])
    office_cols = list(ascii_uppercase[:8])
    col_names = id_vars + candidates + office_cols
    df.columns = col_names
    # forward-fill town values
    filled_towns = df['town'].fillna(method='ffill')
    df = df.assign(town=filled_towns)
    # removing summations
    df = df.dropna()
    # removing extra spaces
    stripped_towns = df['town'].str.replace("\u3000", "")
    df = df.assign(town=stripped_towns)
    # pivoting
    df = df.drop(labels=office_cols, axis=1)
    tidy_df = pd.melt(df,
                      id_vars=id_vars,
                      var_name='candidate_info',
                      value_name='votes'
                     )
    return tidy_df

## Retrieving city and county names from files we previously downloaded

```python
files = [i for i in os.listdir("presidential-2020/") if not i.startswith('.')] # to skip those hidden files
counties = [re.split("\(|\)", f)[1] for f in files]
print(files)
print(counties)
```

## Applying `tidy_dataframe` on presidential spreadsheets

In [18]:
counties = ['宜蘭縣', '彰化縣', '金門縣', '桃園市', '苗栗縣', '臺南市', '雲林縣', '南投縣', '高雄市', '臺北市', '新北市', '花蓮縣', '新竹市', '新竹縣', '基隆市', '連江縣', '嘉義縣', '嘉義市', '屏東縣', '澎湖縣', '臺東縣', '臺中市']
presidential = pd.DataFrame()
for county in counties:
    file_name = "總統-A05-4-候選人得票數一覽表-各投開票所({}).xls".format(county)
    file_url = quote_plus(file_name)
    spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
    # skip those combined cells
    df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')
    tidy_df = tidy_dataframe(df)
    # appending dataframe of each city/county
    tidy_df['county'] = county
    presidential = presidential.append(tidy_df)
    print("Tidying {}".format(file_name))
presidential = presidential.reset_index(drop=True) # reset index for the appended dataframe

Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(宜蘭縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(彰化縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(金門縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(桃園市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(苗栗縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺南市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(雲林縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(南投縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(高雄市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(花蓮縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(基隆市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(連江縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(屏東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(澎湖縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺中市).xls


In [19]:
presidential.head()

,town,village,office,candidate_info,votes,county
0,宜蘭市,民族里,1.0,(1)\n宋楚瑜\n余湘,37,宜蘭縣
1,宜蘭市,民族里,2.0,(1)\n宋楚瑜\n余湘,31,宜蘭縣
2,宜蘭市,建軍里,3.0,(1)\n宋楚瑜\n余湘,19,宜蘭縣
3,宜蘭市,建軍里,4.0,(1)\n宋楚瑜\n余湘,29,宜蘭縣
4,宜蘭市,泰山里,5.0,(1)\n宋楚瑜\n余湘,25,宜蘭縣


In [20]:
presidential.tail()

,town,village,office,candidate_info,votes,county
51673,和平區,梨山里,1845.0,(3)\n蔡英文\n賴清德,132,臺中市
51674,和平區,梨山里,1846.0,(3)\n蔡英文\n賴清德,107,臺中市
51675,和平區,梨山里,1847.0,(3)\n蔡英文\n賴清德,40,臺中市
51676,和平區,平等里,1848.0,(3)\n蔡英文\n賴清德,24,臺中市
51677,和平區,平等里,1849.0,(3)\n蔡英文\n賴清德,102,臺中市


## Define a function `adjust_presidential` to make minor adjustments on `presidential` after appending

In [21]:
def adjust_presidential(df):
    # split candidate information into 2 columns
    candidate_info_df = df['candidate_info'].str.split("\n", expand=True)
    numbers = candidate_info_df[0].str.replace("\(|\)", "")
    candidates = candidate_info_df[1].str.cat(candidate_info_df[2], sep="/")
    # re-arrange columns
    df = df.drop(labels='candidate_info', axis=1)
    df['number'] = numbers
    df['candidate'] = candidates
    df['office'] = df['office'].astype(int)
    df = df[['county', 'town', 'village', 'office', 'number', 'candidate', 'votes']]
    return df

## Applying `adjust_presidential` on `presidential`

In [22]:
presidential = adjust_presidential(presidential)

In [23]:
presidential.head()

,county,town,village,office,number,candidate,votes
0,宜蘭縣,宜蘭市,民族里,1,1,宋楚瑜/余湘,37
1,宜蘭縣,宜蘭市,民族里,2,1,宋楚瑜/余湘,31
2,宜蘭縣,宜蘭市,建軍里,3,1,宋楚瑜/余湘,19
3,宜蘭縣,宜蘭市,建軍里,4,1,宋楚瑜/余湘,29
4,宜蘭縣,宜蘭市,泰山里,5,1,宋楚瑜/余湘,25


In [24]:
presidential.tail()

,county,town,village,office,number,candidate,votes
51673,臺中市,和平區,梨山里,1845,3,蔡英文/賴清德,132
51674,臺中市,和平區,梨山里,1846,3,蔡英文/賴清德,107
51675,臺中市,和平區,梨山里,1847,3,蔡英文/賴清德,40
51676,臺中市,和平區,平等里,1848,3,蔡英文/賴清德,24
51677,臺中市,和平區,平等里,1849,3,蔡英文/賴清德,102


## Done wrangling the presidential data!

![](https://media.giphy.com/media/1sjwSoZLcENCE/giphy.gif)

Source: <https://giphy.com/>

## Applying `tidy_dataframe` on legislative spreadsheets

Types of legislator:
- regional
- plains/mountain indigenous
- at large

## Regional, plains indigenous, and mountain indigenous are voted on candidates, while at large are voted on parties

## Applying `tidy_dataframe` on regional spreadsheets

In [25]:
regional = pd.DataFrame()
for county in counties:
    file_name = "區域立委-A05-2-得票數一覽表({}).xls".format(county)
    file_url = quote_plus(file_name)
    spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
    xl = pd.ExcelFile(spreadsheet_url)
    for sheet in xl.sheet_names:
        # skip those combined cells
        df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',', sheet_name=sheet)
        tidy_df = tidy_dataframe(df)
        # appending dataframe of each city/county
        tidy_df['county'] = county
        tidy_df['electoral_district'] = sheet
        regional = regional.append(tidy_df)
        print("Tidying {} of {}".format(sheet, file_name))
regional = regional.reset_index(drop=True) # reset index for the appended dataframe

Tidying 宜蘭縣選舉區 of 區域立委-A05-2-得票數一覽表(宜蘭縣).xls
Tidying 彰化縣第1選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第2選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第3選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 彰化縣第4選舉區 of 區域立委-A05-2-得票數一覽表(彰化縣).xls
Tidying 金門縣選舉區 of 區域立委-A05-2-得票數一覽表(金門縣).xls
Tidying 桃園市第1選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第2選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第3選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第4選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第5選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 桃園市第6選舉區 of 區域立委-A05-2-得票數一覽表(桃園市).xls
Tidying 苗栗縣第1選舉區 of 區域立委-A05-2-得票數一覽表(苗栗縣).xls
Tidying 苗栗縣第2選舉區 of 區域立委-A05-2-得票數一覽表(苗栗縣).xls
Tidying 臺南市第1選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第2選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第3選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第4選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第5選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 臺南市第6選舉區 of 區域立委-A05-2-得票數一覽表(臺南市).xls
Tidying 雲林縣第1選舉區 of 區域立委-A05-2-得票數一覽表(雲林縣).xls
Tidying 雲林縣第2選舉區 

In [26]:
regional.head()

,town,village,office,candidate_info,votes,county,electoral_district
0,宜蘭市,民族里,1.0,(1)\n黃定和\n無,128,宜蘭縣,宜蘭縣選舉區
1,宜蘭市,民族里,2.0,(1)\n黃定和\n無,138,宜蘭縣,宜蘭縣選舉區
2,宜蘭市,建軍里,3.0,(1)\n黃定和\n無,122,宜蘭縣,宜蘭縣選舉區
3,宜蘭市,建軍里,4.0,(1)\n黃定和\n無,104,宜蘭縣,宜蘭縣選舉區
4,宜蘭市,泰山里,5.0,(1)\n黃定和\n無,129,宜蘭縣,宜蘭縣選舉區


## Define a function `adjust_legislative` to make minor adjustments on `regional` after appending

In [27]:
def adjust_legislative(df):
    # split candidate information into 2 columns
    candidate_info_df = df['candidate_info'].str.split("\n", expand=True)
    numbers = candidate_info_df[0].str.replace("\(|\)", "")
    candidates = candidate_info_df[1]
    parties = candidate_info_df[2]
    # re-arrange columns
    df = df.drop(labels='candidate_info', axis=1)
    df['number'] = numbers
    df['candidate'] = candidates
    df['party'] = parties
    df['office'] = df['office'].astype(int)
    return df

## Applying `adjust_regional` on `regional`

In [28]:
regional = adjust_legislative(regional)
regional = regional[['county', 'electoral_district', 'town', 'village', 'office', 'number', 'party', 'candidate', 'votes']]

In [29]:
regional.head()

,county,electoral_district,town,village,office,number,party,candidate,votes
0,宜蘭縣,宜蘭縣選舉區,宜蘭市,民族里,1,1,無,黃定和,128
1,宜蘭縣,宜蘭縣選舉區,宜蘭市,民族里,2,1,無,黃定和,138
2,宜蘭縣,宜蘭縣選舉區,宜蘭市,建軍里,3,1,無,黃定和,122
3,宜蘭縣,宜蘭縣選舉區,宜蘭市,建軍里,4,1,無,黃定和,104
4,宜蘭縣,宜蘭縣選舉區,宜蘭市,泰山里,5,1,無,黃定和,129


## Applying `tidy_dataframe` on plains indigenous and mountain indigenous spreadsheets

In [30]:
indigenous = pd.DataFrame()
indigenous_types = ['山地', '平地']
for county in counties:
    for indigenous_type in indigenous_types:
        file_name = "{}立委-A05-4-得票數一覽表({}).xls".format(indigenous_type, county)
        file_url = quote_plus(file_name)
        spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
        # skip those combined cells
        df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')
        tidy_df = tidy_dataframe(df)
        # appending dataframe of each city/county
        tidy_df['county'] = county
        tidy_df['electoral_district'] = '{}原住民'.format(indigenous_type)
        indigenous = indigenous.append(tidy_df)
        print("Tidying {}".format(file_name))
indigenous = indigenous.reset_index(drop=True) # reset index for the appended dataframe

Tidying 山地立委-A05-4-得票數一覽表(宜蘭縣).xls
Tidying 平地立委-A05-4-得票數一覽表(宜蘭縣).xls
Tidying 山地立委-A05-4-得票數一覽表(彰化縣).xls
Tidying 平地立委-A05-4-得票數一覽表(彰化縣).xls
Tidying 山地立委-A05-4-得票數一覽表(金門縣).xls
Tidying 平地立委-A05-4-得票數一覽表(金門縣).xls
Tidying 山地立委-A05-4-得票數一覽表(桃園市).xls
Tidying 平地立委-A05-4-得票數一覽表(桃園市).xls
Tidying 山地立委-A05-4-得票數一覽表(苗栗縣).xls
Tidying 平地立委-A05-4-得票數一覽表(苗栗縣).xls
Tidying 山地立委-A05-4-得票數一覽表(臺南市).xls
Tidying 平地立委-A05-4-得票數一覽表(臺南市).xls
Tidying 山地立委-A05-4-得票數一覽表(雲林縣).xls
Tidying 平地立委-A05-4-得票數一覽表(雲林縣).xls
Tidying 山地立委-A05-4-得票數一覽表(南投縣).xls
Tidying 平地立委-A05-4-得票數一覽表(南投縣).xls
Tidying 山地立委-A05-4-得票數一覽表(高雄市).xls
Tidying 平地立委-A05-4-得票數一覽表(高雄市).xls
Tidying 山地立委-A05-4-得票數一覽表(臺北市).xls
Tidying 平地立委-A05-4-得票數一覽表(臺北市).xls
Tidying 山地立委-A05-4-得票數一覽表(新北市).xls
Tidying 平地立委-A05-4-得票數一覽表(新北市).xls
Tidying 山地立委-A05-4-得票數一覽表(花蓮縣).xls
Tidying 平地立委-A05-4-得票數一覽表(花蓮縣).xls
Tidying 山地立委-A05-4-得票數一覽表(新竹市).xls
Tidying 平地立委-A05-4-得票數一覽表(新竹市).xls
Tidying 山地立委-A05-4-得票數一覽表(新竹縣).xls
Tidying 平地立委-A05-4-得票數一覽表(新竹縣).xls
Tidying 山地立委-A05-4-得

In [31]:
indigenous.head()

,town,village,office,candidate_info,votes,county,electoral_district
0,宜蘭市,民族里,1.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民
1,宜蘭市,民族里,2.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民
2,宜蘭市,建軍里,3.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民
3,宜蘭市,建軍里,4.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民
4,宜蘭市,泰山里,5.0,(1)\n伍麗華Saidhai．Tahovecahe\n民主進步黨,0,宜蘭縣,山地原住民


## Applying `adjust_legislative` on `indigenous` to make minor adjustments after appending

In [32]:
indigenous = adjust_legislative(indigenous)
indigenous = indigenous[['county', 'electoral_district', 'town', 'village', 'office', 'number', 'party', 'candidate', 'votes']]

In [33]:
indigenous.head()

,county,electoral_district,town,village,office,number,party,candidate,votes
0,宜蘭縣,山地原住民,宜蘭市,民族里,1,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0
1,宜蘭縣,山地原住民,宜蘭市,民族里,2,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0
2,宜蘭縣,山地原住民,宜蘭市,建軍里,3,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0
3,宜蘭縣,山地原住民,宜蘭市,建軍里,4,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0
4,宜蘭縣,山地原住民,宜蘭市,泰山里,5,1,民主進步黨,伍麗華Saidhai．Tahovecahe,0


## Regional and indigenous are voted on candidates, so probably we can combine them vertically as well

In [34]:
legislative_regional = pd.concat([regional, indigenous], axis=0)
legislative_regional = legislative_regional.reset_index(drop=True)

## Applying `tidy_dataframe` on at large spreadsheets

In [35]:
legislative_at_large = pd.DataFrame()
for county in counties:
    file_name = "不分區立委-A05-6-得票數一覽表({}).xls".format(county)
    file_url = quote_plus(file_name)
    spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
    # skip those combined cells
    df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')
    tidy_df = tidy_dataframe(df)
    # appending dataframe of each city/county
    tidy_df['county'] = county
    legislative_at_large = legislative_at_large.append(tidy_df)
    print("Tidying {}".format(file_name))
legislative_at_large = legislative_at_large.reset_index(drop=True) # reset index for the appended dataframe

Tidying 不分區立委-A05-6-得票數一覽表(宜蘭縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(彰化縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(金門縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(桃園市).xls
Tidying 不分區立委-A05-6-得票數一覽表(苗栗縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺南市).xls
Tidying 不分區立委-A05-6-得票數一覽表(雲林縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(南投縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(高雄市).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺北市).xls
Tidying 不分區立委-A05-6-得票數一覽表(新北市).xls
Tidying 不分區立委-A05-6-得票數一覽表(花蓮縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(新竹市).xls
Tidying 不分區立委-A05-6-得票數一覽表(新竹縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(基隆市).xls
Tidying 不分區立委-A05-6-得票數一覽表(連江縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(嘉義縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(嘉義市).xls
Tidying 不分區立委-A05-6-得票數一覽表(屏東縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(澎湖縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺東縣).xls
Tidying 不分區立委-A05-6-得票數一覽表(臺中市).xls


In [36]:
legislative_at_large.head()

,town,village,office,candidate_info,votes,county
0,宜蘭市,民族里,1.0,(1)\n\n合一行動聯盟,2,宜蘭縣
1,宜蘭市,民族里,2.0,(1)\n\n合一行動聯盟,1,宜蘭縣
2,宜蘭市,建軍里,3.0,(1)\n\n合一行動聯盟,2,宜蘭縣
3,宜蘭市,建軍里,4.0,(1)\n\n合一行動聯盟,0,宜蘭縣
4,宜蘭市,泰山里,5.0,(1)\n\n合一行動聯盟,3,宜蘭縣


## Applying `adjust_legislative` on `legislative_at_large` to make minor adjustments after appending

In [37]:
legislative_at_large = adjust_legislative(legislative_at_large)
legislative_at_large = legislative_at_large[['county', 'town', 'village', 'office', 'number', 'party', 'votes']]
legislative_at_large.head()

,county,town,village,office,number,party,votes
0,宜蘭縣,宜蘭市,民族里,1,1,合一行動聯盟,2
1,宜蘭縣,宜蘭市,民族里,2,1,合一行動聯盟,1
2,宜蘭縣,宜蘭市,建軍里,3,1,合一行動聯盟,2
3,宜蘭縣,宜蘭市,建軍里,4,1,合一行動聯盟,0
4,宜蘭縣,宜蘭市,泰山里,5,1,合一行動聯盟,3


## We can also define a class `TaiwanElection2020` for above wranglings

<https://gist.github.com/yaojenkuo/68164c0ae503c87923c2727ff018646b>

In [38]:
from taiwan_election_2020 import TaiwanElection2020

tw_election = TaiwanElection2020()
presidential = tw_election.generate_presidential()
legislative_regional = tw_election.generate_legislative_regional()
legislative_at_large = tw_election.generate_legislative_at_large()

Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(宜蘭縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(彰化縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(金門縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(桃園市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(苗栗縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺南市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(雲林縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(南投縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(高雄市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新北市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(花蓮縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(新竹縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(基隆市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(連江縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(嘉義市).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(屏東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(澎湖縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺東縣).xls
Tidying 總統-A05-4-候選人得票數一覽表-各投開票所(臺中市).xls
Tidying 宜蘭縣選舉區 of 區域立委-A05-2-得票數一覽表(宜蘭縣).xls
Tidying 彰化縣第1選舉區 of 區域立委-A05-2-

NameError: name 'counties' is not defined

## Done wrangling the legislative data!

![](https://media.giphy.com/media/13tw3BAKGrRMRy/giphy.gif)

Source: <https://giphy.com/>

## Export tidy dataframes as `csv` via `to_csv` method

We usually set `index=False` as we do not use `Index` in flat files.

In [39]:
presidential.to_csv('presidential.csv', index=False)
legislative_regional.to_csv('legislative_regional.csv', index=False)
legislative_at_large.to_csv('legislative_at_large.csv', index=False)

## Export tidy dataframes as `sqlite` tables via `to_sql` method and `sqlite3` connection

We usually set `index=False` as we do not include `Index` in tables as a column.

In [40]:
conn = sqlite3.connect('tw-election-2020.db')
presidential.to_sql('presidential', con=conn, index=False, if_exists='replace')
legislative_regional.to_sql('legislative_regional', con=conn, index=False, if_exists='replace')
legislative_at_large.to_sql('legislative_at_large', con=conn, index=False, if_exists='replace')